In [ ]:
import pandas as pd
import wandb
api = wandb.Api()
from consts import BATCHSIZE, all_collections
from consts import cnn_group_data_extra_features_no_leak as cnn_group_data

databases = ["raw","meters","exponential","all_collections","all_spaces"]
spaces = ["indoor","garage","outdoor"]

model_dict:dict = {}

# Project is specified by <entity/project-name>
for group in cnn_group_data.keys():
    runs = api.runs(
    "santi-mbenites-university-of-lisbon/wifi-rssi-gradient-search",
    filters={"group":group}
    )
    print(f"{group} : {len(runs)}")
    for run in runs:
        model_dict[group] = model_dict.get(group,[])
        model_dict[group].append(run.name)


CNN_DATA_ANALYSIS_reto_grande_outdoor_extra_features : 3
CNN_DATA_ANALYSIS_reto_grande_indoor_extra_features : 3
CNN_DATA_ANALYSIS_reto_grande_garage_extra_features : 3
CNN_DATA_ANALYSIS_all_spaces_reto_grande_extra_features : 3
CNN_DATA_ANALYSIS_all_collections_indoor_outdoor_extra_features : 3
CNN_DATA_ANALYSIS_all_collections_indoor_garage_extra_features : 3
CNN_DATA_ANALYSIS_all_collections_garage_outdoor_extra_features : 3
CNN_DATA_ANALYSIS_all_collections_garage_outdoor_indoor_extra_features : 3


In [17]:
display(model_dict)

{'CNN_DATA_ANALYSIS_reto_grande_outdoor_extra_features': ['reto_grande_outdoor_extra_features_run0_depth0_model4',
  'reto_grande_outdoor_extra_features_run1_depth1_model2',
  'reto_grande_outdoor_extra_features_run2_depth0_model3'],
 'CNN_DATA_ANALYSIS_reto_grande_indoor_extra_features': ['reto_grande_indoor_extra_features_run0_depth1_model5',
  'reto_grande_indoor_extra_features_run1_depth4_model4',
  'reto_grande_indoor_extra_features_run2_depth4_model1'],
 'CNN_DATA_ANALYSIS_reto_grande_garage_extra_features': ['reto_grande_garage_extra_features_run0_depth3_model1',
  'reto_grande_garage_extra_features_run1_depth3_model4',
  'reto_grande_garage_extra_features_run2_depth2_model0'],
 'CNN_DATA_ANALYSIS_all_spaces_reto_grande_extra_features': ['all_spaces_reto_grande_extra_features_run0_depth3_model5',
  'all_spaces_reto_grande_extra_features_run1_depth2_model7',
  'all_spaces_reto_grande_extra_features_run2_depth4_model0'],
 'CNN_DATA_ANALYSIS_all_collections_indoor_outdoor_extra_fea

In [18]:
import os
from model_evaluation import evaluate_model_on_collections
from feature_lists import DATASET_TO_FEATURE

os.environ["CUDA_VISIBLE_DEVICES"] = "5"

results = {}
for group in cnn_group_data:
    results[group] = []


for current_group,all_runs in model_dict.items():

    for current_run in all_runs:
        current_group_data      = cnn_group_data[current_group]
        current_model_storage   = current_group_data["model_storage"]
        current_database        = current_group_data["database"]
        current_model_path      = f"{current_model_storage}/{current_run}.pt"
        feature_list            = DATASET_TO_FEATURE[current_database]
        relative_coeficient     = current_group_data["relative_coeficient"]
        model_output_name       = current_run
        val_collection          = current_group_data["validation_collections"]

        for collections_name, current_collections in val_collection.items(): 
            
            if not os.path.exists(current_model_path):
                print(f"Model doesnt exist {model_output_name} with path {current_model_path}")
                continue
            
            try:
                mae, mse, rmse, n = evaluate_model_on_collections(
                    current_model_path, current_collections, current_database, BATCHSIZE, None, feature_list
                )
            except Exception as e:
                print(f"Failed Processing {model_output_name} with path {current_model_path}")
                print(e)
                continue
            mae_final = mae * relative_coeficient
            mse_final = mse * relative_coeficient
            rmse_final = rmse * relative_coeficient
            print(f"{model_output_name} | Collections : {collections_name} | N={n} | MAE={mae_final:.6f} | MSE={mse_final:.6f} | RMSE={rmse_final:.6f}")
            results[current_group].append((model_output_name, collections_name, mae_final, mse_final, rmse_final))




reto_grande_outdoor_extra_features_run0_depth0_model4 | Collections : reto_grande_outdoor | N=19550 | MAE=0.266075 | MSE=0.004132 | RMSE=0.363608
reto_grande_outdoor_extra_features_run0_depth0_model4 | Collections : reto_grande_indoor | N=12243 | MAE=17.628785 | MSE=11.560945 | RMSE=19.234091
reto_grande_outdoor_extra_features_run0_depth0_model4 | Collections : reto_grande_garage | N=22621 | MAE=6.071890 | MSE=1.639554 | RMSE=7.243323
reto_grande_outdoor_extra_features_run0_depth0_model4 | Collections : all_reto_grande | N=54414 | MAE=6.586195 | MSE=3.284243 | RMSE=10.251623
reto_grande_outdoor_extra_features_run1_depth1_model2 | Collections : reto_grande_outdoor | N=19550 | MAE=0.525395 | MSE=0.020619 | RMSE=0.812277
reto_grande_outdoor_extra_features_run1_depth1_model2 | Collections : reto_grande_indoor | N=12243 | MAE=15.639283 | MSE=13.555466 | RMSE=20.827264
reto_grande_outdoor_extra_features_run1_depth1_model2 | Collections : reto_grande_garage | N=22621 | MAE=11.115105 | MSE=5.6

In [19]:
import pandas as pd
for group, rows in results.items():

    df = pd.DataFrame(rows, columns=["model","collections", "mae", "mse", "rmse"])
    print(f"GROUP : {group}")
    styled = df.style.background_gradient(cmap="RdYlGn_r")  # _r reverses so green=low, red=high
    display(styled)

GROUP : CNN_DATA_ANALYSIS_reto_grande_outdoor_extra_features


,model,collections,mae,mse,rmse
0,reto_grande_outdoor_extra_features_run0_depth0_model4,reto_grande_outdoor,0.266075,0.004132,0.363608
1,reto_grande_outdoor_extra_features_run0_depth0_model4,reto_grande_indoor,17.628785,11.560945,19.234091
2,reto_grande_outdoor_extra_features_run0_depth0_model4,reto_grande_garage,6.071890,1.639554,7.243323
3,reto_grande_outdoor_extra_features_run0_depth0_model4,all_reto_grande,6.586195,3.284243,10.251623
4,reto_grande_outdoor_extra_features_run1_depth1_model2,reto_grande_outdoor,0.525395,0.020619,0.812277
5,reto_grande_outdoor_extra_features_run1_depth1_model2,reto_grande_indoor,15.639283,13.555466,20.827264
6,reto_grande_outdoor_extra_features_run1_depth1_model2,reto_grande_garage,11.115105,5.600985,13.387737
7,reto_grande_outdoor_extra_features_run1_depth1_model2,all_reto_grande,8.328334,5.385793,13.128038
8,reto_grande_outdoor_extra_features_run2_depth0_model3,reto_grande_outdoor,0.190054,0.002662,0.291852
9,reto_grande_outdoor_extra_features_run2_depth0_model3,reto_grande_indoor,25.468084,21.482027,26.218789


GROUP : CNN_DATA_ANALYSIS_reto_grande_indoor_extra_features


,model,collections,mae,mse,rmse
0,reto_grande_indoor_extra_features_run0_depth1_model5,reto_grande_outdoor,9.725780,4.494859,11.993143
1,reto_grande_indoor_extra_features_run0_depth1_model5,reto_grande_indoor,0.000032,0.000000,0.000032
2,reto_grande_indoor_extra_features_run0_depth1_model5,reto_grande_garage,9.872539,4.643511,12.189846
3,reto_grande_indoor_extra_features_run0_depth1_model5,all_reto_grande,7.598524,3.545326,10.651311
4,reto_grande_indoor_extra_features_run1_depth4_model4,reto_grande_outdoor,9.725865,4.494963,11.993283
5,reto_grande_indoor_extra_features_run1_depth4_model4,reto_grande_indoor,0.000272,0.000000,0.000903
6,reto_grande_indoor_extra_features_run1_depth4_model4,reto_grande_garage,9.872622,4.643654,12.190033
7,reto_grande_indoor_extra_features_run1_depth4_model4,all_reto_grande,7.598643,3.545423,10.651456
8,reto_grande_indoor_extra_features_run2_depth4_model1,reto_grande_outdoor,9.725799,4.494888,11.993183
9,reto_grande_indoor_extra_features_run2_depth4_model1,reto_grande_indoor,0.000174,0.000000,0.000224


GROUP : CNN_DATA_ANALYSIS_reto_grande_garage_extra_features


...

,model,collections,mae,mse,rmse
0,all_collections_indoor_garage_extra_features_run0_depth3_model4,all_collections_garage,0.399950,0.012512,0.632771
1,all_collections_indoor_garage_extra_features_run0_depth3_model4,all_collections_outdoor,4.837575,1.335249,6.536663
2,all_collections_indoor_garage_extra_features_run0_depth3_model4,all_collections_indoor,0.049679,0.000427,0.116871
3,all_collections_indoor_garage_extra_features_run0_depth3_model4,all_collections_garage_outdoor,3.419260,0.912487,5.403665
4,all_collections_indoor_garage_extra_features_run0_depth3_model4,all_collections_garage_indoor,0.305773,0.009263,0.544442
5,all_collections_indoor_garage_extra_features_run0_depth3_model4,all_collections_indoor_outdoor,4.132315,1.138629,6.036233
6,all_collections_indoor_garage_extra_features_run0_depth3_model4,all_collections_garage_outdoor_indoor,3.064870,0.816563,5.111753
7,all_collections_indoor_garage_extra_features_run1_depth0_model2,all_collections_garage,0.847839,0.043656,1.181945
8,all_collections_indoor_garage_extra_features_run1_depth0_model2,all_collections_outdoor,4.968922,1.309378,6.473029
9,all_collections_indoor_garage_extra_features_run1_depth0_model2,all_collections_indoor,0.213435,0.003382,0.328961


GROUP : CNN_DATA_ANALYSIS_all_collections_garage_outdoor_extra_features


,model,collections,mae,mse,rmse
0,all_collections_garage_outdoor_extra_features_run0_depth0_model3,all_collections_garage,1.854975,0.216756,2.633662
1,all_collections_garage_outdoor_extra_features_run0_depth0_model3,all_collections_outdoor,1.711484,0.185951,2.439352
2,all_collections_garage_outdoor_extra_features_run0_depth0_model3,all_collections_indoor,16.227796,9.749044,17.662655
3,all_collections_garage_outdoor_extra_features_run0_depth0_model3,all_collections_garage_outdoor,1.757345,0.195797,2.503096
4,all_collections_garage_outdoor_extra_features_run0_depth0_model3,all_collections_garage_indoor,5.719372,2.779686,9.431328
5,all_collections_garage_outdoor_extra_features_run0_depth0_model3,all_collections_indoor_outdoor,3.849748,1.594602,7.143337
6,all_collections_garage_outdoor_extra_features_run0_depth0_model3,all_collections_garage_outdoor_indoor,3.279249,1.200542,6.198173
7,all_collections_garage_outdoor_extra_features_run1_depth4_model3,all_collections_garage,0.575093,0.019600,0.791964
8,all_collections_garage_outdoor_extra_features_run1_depth4_model3,all_collections_outdoor,0.411433,0.009873,0.562073
9,all_collections_garage_outdoor_extra_features_run1_depth4_model3,all_collections_indoor,22.408507,26.913214,29.346598


GROUP : CNN_DATA_ANALYSIS_all_collections_garage_outdoor_indoor_extra_features


,model,collections,mae,mse,rmse
0,all_collections_garage_outdoor_indoor_extra_features_run0_depth3_model0,all_collections_garage,0.137783,0.001336,0.206752
1,all_collections_garage_outdoor_indoor_extra_features_run0_depth3_model0,all_collections_outdoor,0.126806,0.001085,0.186356
2,all_collections_garage_outdoor_indoor_extra_features_run0_depth3_model0,all_collections_indoor,0.027154,0.000055,0.042072
3,all_collections_garage_outdoor_indoor_extra_features_run0_depth3_model0,all_collections_garage_outdoor,0.130314,0.001165,0.193109
4,all_collections_garage_outdoor_indoor_extra_features_run0_depth3_model0,all_collections_garage_indoor,0.108038,0.000992,0.178127
5,all_collections_garage_outdoor_indoor_extra_features_run0_depth3_model0,all_collections_indoor_outdoor,0.112127,0.000934,0.172841
6,all_collections_garage_outdoor_indoor_extra_features_run0_depth3_model0,all_collections_garage_outdoor_indoor,0.119464,0.001049,0.183181
7,all_collections_garage_outdoor_indoor_extra_features_run1_depth0_model5,all_collections_garage,0.175366,0.002035,0.255159
8,all_collections_garage_outdoor_indoor_extra_features_run1_depth0_model5,all_collections_outdoor,0.142505,0.001261,0.200904
9,all_collections_garage_outdoor_indoor_extra_features_run1_depth0_model5,all_collections_indoor,0.044937,0.000100,0.056581


In [20]:
import re
import pandas as pd
import matplotlib.pyplot as plt


combined_rows = []

for group, rows in results.items():
    
    dataset = max((d for d in databases if d in group), key=len, default=None)
    space = max((s for s in spaces if s in group), key=len, default=None)
    
    for current_row in rows:
        print(current_row)
        (model_output_name, collections_name, mae_final, mse_final, rmse_final) = current_row
        combined_rows.append((group,collections_name,mae_final,mse_final,rmse_final))

print(combined_rows)

('reto_grande_outdoor_extra_features_run0_depth0_model4', 'reto_grande_outdoor', 0.26607497859184087, 0.00413158189071833, 0.36360778388668547)
('reto_grande_outdoor_extra_features_run0_depth0_model4', 'reto_grande_indoor', 17.628784939947348, 11.56094492399974, 19.234090505349915)
('reto_grande_outdoor_extra_features_run0_depth0_model4', 'reto_grande_garage', 6.07189020265883, 1.6395538171592638, 7.243322590434341)
('reto_grande_outdoor_extra_features_run0_depth0_model4', 'all_reto_grande', 6.586194555065477, 3.284242680664559, 10.251622592607761)
('reto_grande_outdoor_extra_features_run1_depth1_model2', 'reto_grande_outdoor', 0.5253946909087394, 0.020618564733458907, 0.8122770903273617)
('reto_grande_outdoor_extra_features_run1_depth1_model2', 'reto_grande_indoor', 15.639282599058667, 13.555466206588067, 20.82726382919317)
('reto_grande_outdoor_extra_features_run1_depth1_model2', 'reto_grande_garage', 11.115104566860524, 5.6009846277912745, 13.387737228124879)
('reto_grande_outdoor_e

In [21]:
import pandas as pd

singles = {
    "CNN_DATA_ANALYSIS_reto_grande_garage_exponential"          : "reto_grande_garage_exponential",
    "CNN_DATA_ANALYSIS_reto_grande_indoor_exponential"          : "reto_grande_indoor_exponential",
    "CNN_DATA_ANALYSIS_reto_grande_outdoor_exponential"         : "reto_grande_outdoor_exponential",
    "CNN_DATA_ANALYSIS_reto_grande_garage_raw"                  : "reto_grande_garage_raw",
    "CNN_DATA_ANALYSIS_reto_grande_indoor_raw"                  : "reto_grande_indoor_raw",
    "CNN_DATA_ANALYSIS_reto_grande_outdoor_raw"                 : "reto_grande_outdoor_raw",
    "CNN_DATA_ANALYSIS_reto_grande_garage_meters"               : "reto_grande_garage_meters",
    "CNN_DATA_ANALYSIS_reto_grande_indoor_meters"               : "reto_grande_indoor_meters",
    "CNN_DATA_ANALYSIS_reto_grande_outdoor_meters"              : "reto_grande_outdoor_meters",
    "CNN_DATA_ANALYSIS_all_spaces_reto_grande"                  : "all_spaces_reto_grande",
    "CNN_DATA_ANALYSIS_reto_grande_outdoor_extra_features"      : "reto_grande_outdoor_extra_features", 
    "CNN_DATA_ANALYSIS_reto_grande_indoor_extra_features"       : "reto_grande_indoor_extra_features", 
    "CNN_DATA_ANALYSIS_reto_grande_garage_extra_features"       : "reto_grande_garage_extra_features", 
    "CNN_DATA_ANALYSIS_all_spaces_reto_grande_extra_features"   : "all_spaces_reto_grande_extra_features", 
    "CNN_DATA_ANALYSIS_reto_grande_outdoor_extra_features_no_leak"      : "reto_grande_outdoor_extra_features_no_leak", 
    "CNN_DATA_ANALYSIS_reto_grande_indoor_extra_features_no_leak"       : "reto_grande_indoor_extra_features_no_leak", 
    "CNN_DATA_ANALYSIS_reto_grande_garage_extra_features_no_leak"       : "reto_grande_garage_extra_features_no_leak", 
    "CNN_DATA_ANALYSIS_all_spaces_reto_grande_extra_features_no_leak"   : "all_spaces_reto_grande_extra_features_no_leak", 
}

spaces = {
    "CNN_DATA_ANALYSIS_all_collections_garage_outdoor"                      : "all_coll_garage_outdoor",
    "CNN_DATA_ANALYSIS_all_collections_indoor_garage"                       : "all_coll_indoor_garage",
    "CNN_DATA_ANALYSIS_all_collections_indoor_outdoor"                      : "all_coll_indoor_outdoor",
    "CNN_DATA_ANALYSIS_all_collections_garage_outdoor_indoor"               : "all_coll_garage_outdoor_indoor",
    "CNN_DATA_ANALYSIS_all_collections_indoor_outdoor_extra_features"       : "all_coll_indoor_outdoor_extra_features",
    "CNN_DATA_ANALYSIS_all_collections_indoor_garage_extra_features"        : "all_coll_indoor_garage_extra_features",
    "CNN_DATA_ANALYSIS_all_collections_garage_outdoor_extra_features"       : "all_coll_garage_outdoor_extra_features",
    "CNN_DATA_ANALYSIS_all_collections_garage_outdoor_indoor_extra_features": "all_coll_garage_outdoor_indoor_extra_features",
    "CNN_DATA_ANALYSIS_all_collections_indoor_outdoor_extra_features_no_leak"       : "all_coll_indoor_outdoor_extra_features_no_leak",
    "CNN_DATA_ANALYSIS_all_collections_indoor_garage_extra_features_no_leak"        : "all_coll_indoor_garage_extra_features_no_leak",
    "CNN_DATA_ANALYSIS_all_collections_garage_outdoor_extra_features_no_leak"       : "all_coll_garage_outdoor_extra_features_no_leak",
    "CNN_DATA_ANALYSIS_all_collections_garage_outdoor_indoor_extra_features_no_leak": "all_coll_garage_outdoor_indoor_extra_features_no_leak",
}

spaces_datasets = []
singles_datasets = []

for row in combined_rows:
    (group,collections_name,mae_final,mse_final,rmse_final) = row
    if group in spaces:
        group_defnition = spaces[group]
        spaces_datasets.append((group_defnition,collections_name,mae_final,mse_final,rmse_final))
    elif group in singles:
        group_defnition = singles[group]
        singles_datasets.append((group_defnition,collections_name,mae_final,mse_final,rmse_final))
    else:
        print(f"ERROR : {group}")

print(spaces_datasets)
print(singles_datasets)


[('all_coll_indoor_outdoor_extra_features', 'all_collections_garage', 4.314270145956648, 1.0071575984330983, 5.677062898177115), ('all_coll_indoor_outdoor_extra_features', 'all_collections_outdoor', 0.08068078980745845, 0.0003889823138953968, 0.11156806910874051), ('all_coll_indoor_outdoor_extra_features', 'all_collections_indoor', 0.03960327545915671, 0.00011095796641583333, 0.05958737219668834), ('all_coll_indoor_outdoor_extra_features', 'all_collections_garage_outdoor', 1.4337836530994943, 0.3221635696781067, 3.210799624657293), ('all_coll_indoor_outdoor_extra_features', 'all_collections_garage_indoor', 3.1649474992255713, 0.7363946179329508, 4.85434112664679), ('all_coll_indoor_outdoor_extra_features', 'all_collections_indoor_outdoor', 0.07463004101131548, 0.0003480291159591229, 0.10553166212417926), ('all_coll_indoor_outdoor_extra_features', 'all_collections_garage_outdoor_indoor', 1.2871532163302013, 0.2882922586221177, 3.0373265013672412), ('all_coll_indoor_outdoor_extra_feature

In [22]:
import pandas as pd

# rows: list[tuple] like (dataset, collections, mae, mse, rmse)
def make_wide_by_collection(rows, *,
                            caption="Modelos - erros médios por coleção",
                            label="tab:model_metrics_by_collection"):
    # Build DataFrame
    df = pd.DataFrame(rows, columns=["dataset", "collections", "mae", "mse", "rmse"])

    if df.empty:
        print("No data provided.")
        return None, None, None

    # Robust pivot (handles duplicate (dataset, collections) by averaging)
    df_wide = (
        df.pivot_table(index="dataset",
                       columns="collections",
                       values=["mae", "mse", "rmse"],
                       aggfunc="mean")
          .swaplevel(0, 1, axis=1)                 # put collection on top, metrics underneath
          .sort_index(axis=1, level=[0, 1])        # nice column order
          .sort_index(axis=0)                      # sort datasets
    )

    # Optional: round displayed values only
    styled = (
        df_wide.style
              .format(precision=2, decimal=",")
              .background_gradient(cmap="RdYlGn_r")
    )

    latex = styled.to_latex(
        caption=caption,
        label=label,
        position="H",
        environment="table",
        convert_css=True,
        multicol_align="c",
    )

    return df, df_wide, (styled, latex.replace("_", "\\_"))


def latex_with_lines(df, *args, **kwargs):
    kwargs['column_format'] = '|'.join([''] + ['l'] * df.index.nlevels
                                            + ['r'] * df.shape[1] + [''])
    res = df.to_latex(*args, **kwargs)
    return res.replace('\\\\\n', '\\\\ \\midrule\n')

In [23]:
# Map collections labels
CNN_space_combinations = {
    "all_collections_garage"                : "garage",
    "all_collections_outdoor"               : "outdoor",
    "all_collections_indoor"                : "indoor",
    "all_collections_garage_outdoor"        : "garage_outdoor",
    "all_collections_garage_indoor"         : "garage_indoor",
    "all_collections_indoor_outdoor"        : "indoor_outdoor",
    "all_collections_garage_outdoor_indoor" : "garage_outdoor_indoor",
}

# Build DataFrame
df = pd.DataFrame(spaces_datasets, columns=["dataset", "collections", "mae", "mse", "rmse"])

# Robust pivot (handles duplicate (dataset, collections) by averaging)
df_wide = (
    df.pivot_table(index="dataset",
                    columns="collections",
                    values=["mae", "mse", "rmse"],
                    aggfunc="mean")
        .swaplevel(0, 1, axis=1)                 # put collection on top, metrics underneath
        .sort_index(axis=1, level=[0, 1])        # nice column order
        .sort_index(axis=0)                      # sort datasets
)

df_wide = df_wide.sort_values(by="dataset", key=lambda x: x.str.len())
df_wide = df_wide.transpose()

# Optional: round displayed values only
styled = (
    df_wide.style
            .format(precision=2, decimal=",")
            .background_gradient(cmap="RdYlGn_r")
)

latex = styled.to_latex(
    caption="Dataset Combination errors",
    label="tb:model_metrics_by_collection_spaces",
    position="H",
    environment="table",
    convert_css=True,
    multicol_align="c",
    hrules=True,
)


display(styled)
print(latex.replace("_", "\\_"))




\begin{table}[H]
\caption{Dataset Combination errors}
\label{tb:model\_metrics\_by\_collection\_spaces}
\begin{tabular}{llrrrr}
\toprule
 & dataset & all\_coll\_indoor\_garage\_extra\_features & all\_coll\_garage\_outdoor\_extra\_features & all\_coll\_indoor\_outdoor\_extra\_features & all\_coll\_garage\_outdoor\_indoor\_extra\_features \\
collections &  &  &  &  &  \\
\midrule
\multirow[c]{3}{*}{all\_collections\_garage} & mae & {\cellcolor[HTML]{39A758}} \color[HTML]{F1F1F1} 0,99 & {\cellcolor[HTML]{0B7D42}} \color[HTML]{F1F1F1} 0,99 & {\cellcolor[HTML]{F8864F}} \color[HTML]{F1F1F1} 5,03 & {\cellcolor[HTML]{FBA05B}} \color[HTML]{000000} 0,27 \\
 & mse & {\cellcolor[HTML]{026C39}} \color[HTML]{F1F1F1} 0,07 & {\cellcolor[HTML]{006837}} \color[HTML]{F1F1F1} 0,09 & {\cellcolor[HTML]{6BBF64}} \color[HTML]{000000} 1,39 & {\cellcolor[HTML]{036E3A}} \color[HTML]{F1F1F1} 0,01 \\
 & rmse & {\cellcolor[HTML]{63BC62}} \color[HTML]{F1F1F1} 1,36 & {\cellcolor[HTML]{108647}} \color[HTML]{F1F1F1} 1,

In [24]:
# Build DataFrame
df = pd.DataFrame(singles_datasets, columns=["model", "collections", "mae", "mse", "rmse"])

# Map collections labels
CNN_space_combinations = {
    "all_collections_garage"                : "garage",
    "all_collections_outdoor"               : "outdoor",
    "all_collections_indoor"                : "indoor",
    "all_collections_garage_outdoor"        : "garage_outdoor",
    "all_collections_garage_indoor"         : "garage_indoor",
    "all_collections_indoor_outdoor"        : "indoor_outdoor",
    "all_collections_garage_outdoor_indoor" : "garage_outdoor_indoor",
}
#df["collections"] = df["collections"].map(CNN_space_combinations)

# Ensure numeric cols are floats (in case they came as strings with commas)
num_cols = ["mae", "mse", "rmse"]
for c in num_cols:
    df[c] = pd.to_numeric(df[c].astype(str).str.replace(",", ".", regex=False), errors="coerce")

# Aggregate and create a MultiIndex (dataset -> collections)
df = (
    df.groupby(["model", "collections"], as_index=False)[num_cols]
      .mean()
      .sort_values(["model", "collections"])
      .set_index(["model", "collections"])   # <- multirow index for rows
)

# Styled display in notebook (keeps the MultiIndex)
styled = (
    df.style
      .format(precision=2, decimal=",")
      .background_gradient(cmap="RdYlGn_r", subset=num_cols)
)

display(styled)

latex = styled.to_latex(
    caption=f"{dataset} error",
    label=f"tb:cnn_all_datasets_{dataset}",
    position="H",
    environment="table",
    convert_css=True,
    multicol_align="c",
    hrules=True,
    # multirow=True,  # uncomment if your pandas version exposes this arg on Styler.to_latex
)

print(latex.replace("_", "\\_"))


\begin{table}[H]
\caption{all\_collections error}
\label{tb:cnn\_all\_datasets\_all\_collections}
\begin{tabular}{llrrr}
\toprule
 &  & mae & mse & rmse \\
model & collections &  &  &  \\
\midrule
\multirow[c]{4}{*}{all\_spaces\_reto\_grande\_extra\_features} & all\_reto\_grande & {\cellcolor[HTML]{0C7F43}} \color[HTML]{F1F1F1} 0,95 & {\cellcolor[HTML]{016A38}} \color[HTML]{F1F1F1} 0,11 & {\cellcolor[HTML]{108647}} \color[HTML]{F1F1F1} 1,39 \\
 & reto\_grande\_garage & {\cellcolor[HTML]{128A49}} \color[HTML]{F1F1F1} 1,45 & {\cellcolor[HTML]{036E3A}} \color[HTML]{F1F1F1} 0,19 & {\cellcolor[HTML]{15904C}} \color[HTML]{F1F1F1} 1,86 \\
 & reto\_grande\_indoor & {\cellcolor[HTML]{006837}} \color[HTML]{F1F1F1} 0,08 & {\cellcolor[HTML]{006837}} \color[HTML]{F1F1F1} 0,00 & {\cellcolor[HTML]{016A38}} \color[HTML]{F1F1F1} 0,11 \\
 & reto\_grande\_outdoor & {\cellcolor[HTML]{0C7F43}} \color[HTML]{F1F1F1} 0,92 & {\cellcolor[HTML]{016A38}} \color[HTML]{F1F1F1} 0,08 & {\cellcolor[HTML]{0D8044}} \col

In [25]:



# Build DataFrame
df = pd.DataFrame(singles_datasets, columns=["dataset", "collections", "mae", "mse", "rmse"])
#display(df)

datasets = list(df["collections"].unique())
for dataset in datasets:
    print(f"Dataset: {dataset}")
    df_current = df.loc[df['collections'] == dataset]

    num_cols = ['mae', 'mse', 'rmse']
    df_current = (
        df_current.groupby('dataset', as_index=False)[num_cols]
        .mean()
    )
    df_current = df_current.sort_values('dataset')
    styled = (
        df_current.style
                .format(precision=2, decimal=",")
                .background_gradient(cmap="RdYlGn_r")
    )

    latex = styled.to_latex(
        caption=f"{dataset} error",
        label=f"tb:cnn_all_datasets_{dataset}",
        position="H",
        environment="table",
        convert_css=True,
        multicol_align="c",
        hrules=True,
    )
    

    display(styled)
    print(latex.replace("_", "\\_"))




Dataset: reto_grande_outdoor


,dataset,mae,mse,rmse
0,all_spaces_reto_grande_extra_features,"0,92","0,08","1,18"
1,reto_grande_garage_extra_features,"5,04","1,25","6,32"
2,reto_grande_indoor_extra_features,"9,73","4,49","11,99"
3,reto_grande_outdoor_extra_features,"0,33","0,01","0,49"


\begin{table}[H]
\caption{reto\_grande\_outdoor error}
\label{tb:cnn\_all\_datasets\_reto\_grande\_outdoor}
\begin{tabular}{llrrr}
\toprule
 & dataset & mae & mse & rmse \\
\midrule
0 & all\_spaces\_reto\_grande\_extra\_features & {\cellcolor[HTML]{108647}} \color[HTML]{F1F1F1} 0,92 & {\cellcolor[HTML]{036E3A}} \color[HTML]{F1F1F1} 0,08 & {\cellcolor[HTML]{0F8446}} \color[HTML]{F1F1F1} 1,18 \\
1 & reto\_grande\_garage\_extra\_features & {\cellcolor[HTML]{FFFEBE}} \color[HTML]{000000} 5,04 & {\cellcolor[HTML]{98D368}} \color[HTML]{000000} 1,25 & {\cellcolor[HTML]{FFFDBC}} \color[HTML]{000000} 6,32 \\
2 & reto\_grande\_indoor\_extra\_features & {\cellcolor[HTML]{A50026}} \color[HTML]{F1F1F1} 9,73 & {\cellcolor[HTML]{A50026}} \color[HTML]{F1F1F1} 4,49 & {\cellcolor[HTML]{A50026}} \color[HTML]{F1F1F1} 11,99 \\
3 & reto\_grande\_outdoor\_extra\_features & {\cellcolor[HTML]{006837}} \color[HTML]{F1F1F1} 0,33 & {\cellcolor[HTML]{006837}} \color[HTML]{F1F1F1} 0,01 & {\cellcolor[HTML]{006837}} 

,dataset,mae,mse,rmse
0,all_spaces_reto_grande_extra_features,"0,08","0,00","0,11"
1,reto_grande_garage_extra_features,"13,05","6,96","14,59"
2,reto_grande_indoor_extra_features,"0,00","0,00","0,00"
3,reto_grande_outdoor_extra_features,"19,58","15,53","22,09"


\begin{table}[H]
\caption{reto\_grande\_indoor error}
\label{tb:cnn\_all\_datasets\_reto\_grande\_indoor}
\begin{tabular}{llrrr}
\toprule
 & dataset & mae & mse & rmse \\
\midrule
0 & all\_spaces\_reto\_grande\_extra\_features & {\cellcolor[HTML]{006837}} \color[HTML]{F1F1F1} 0,08 & {\cellcolor[HTML]{006837}} \color[HTML]{F1F1F1} 0,00 & {\cellcolor[HTML]{016A38}} \color[HTML]{F1F1F1} 0,11 \\
1 & reto\_grande\_garage\_extra\_features & {\cellcolor[HTML]{FDBF6F}} \color[HTML]{000000} 13,05 & {\cellcolor[HTML]{EBF7A3}} \color[HTML]{000000} 6,96 & {\cellcolor[HTML]{FDC171}} \color[HTML]{000000} 14,59 \\
2 & reto\_grande\_indoor\_extra\_features & {\cellcolor[HTML]{006837}} \color[HTML]{F1F1F1} 0,00 & {\cellcolor[HTML]{006837}} \color[HTML]{F1F1F1} 0,00 & {\cellcolor[HTML]{006837}} \color[HTML]{F1F1F1} 0,00 \\
3 & reto\_grande\_outdoor\_extra\_features & {\cellcolor[HTML]{A50026}} \color[HTML]{F1F1F1} 19,58 & {\cellcolor[HTML]{A50026}} \color[HTML]{F1F1F1} 15,53 & {\cellcolor[HTML]{A50026}}

,dataset,mae,mse,rmse
0,all_spaces_reto_grande_extra_features,"1,45","0,19","1,86"
1,reto_grande_garage_extra_features,"0,78","0,07","1,06"
2,reto_grande_indoor_extra_features,"9,87","4,64","12,19"
3,reto_grande_outdoor_extra_features,"9,02","3,76","10,67"


\begin{table}[H]
\caption{reto\_grande\_garage error}
\label{tb:cnn\_all\_datasets\_reto\_grande\_garage}
\begin{tabular}{llrrr}
\toprule
 & dataset & mae & mse & rmse \\
\midrule
0 & all\_spaces\_reto\_grande\_extra\_features & {\cellcolor[HTML]{128A49}} \color[HTML]{F1F1F1} 1,45 & {\cellcolor[HTML]{06733D}} \color[HTML]{F1F1F1} 0,19 & {\cellcolor[HTML]{128A49}} \color[HTML]{F1F1F1} 1,86 \\
1 & reto\_grande\_garage\_extra\_features & {\cellcolor[HTML]{006837}} \color[HTML]{F1F1F1} 0,78 & {\cellcolor[HTML]{006837}} \color[HTML]{F1F1F1} 0,07 & {\cellcolor[HTML]{006837}} \color[HTML]{F1F1F1} 1,06 \\
2 & reto\_grande\_indoor\_extra\_features & {\cellcolor[HTML]{A50026}} \color[HTML]{F1F1F1} 9,87 & {\cellcolor[HTML]{A50026}} \color[HTML]{F1F1F1} 4,64 & {\cellcolor[HTML]{A50026}} \color[HTML]{F1F1F1} 12,19 \\
3 & reto\_grande\_outdoor\_extra\_features & {\cellcolor[HTML]{D22B27}} \color[HTML]{F1F1F1} 9,02 & {\cellcolor[HTML]{F26841}} \color[HTML]{F1F1F1} 3,76 & {\cellcolor[HTML]{E14430}} \c

,dataset,mae,mse,rmse
0,all_spaces_reto_grande_extra_features,"0,95","0,11","1,39"
1,reto_grande_garage_extra_features,"5,07","2,05","8,01"
2,reto_grande_indoor_extra_features,"7,60","3,55","10,65"
3,reto_grande_outdoor_extra_features,"8,27","5,06","12,61"


\begin{table}[H]
\caption{all\_reto\_grande error}
\label{tb:cnn\_all\_datasets\_all\_reto\_grande}
\begin{tabular}{llrrr}
\toprule
 & dataset & mae & mse & rmse \\
\midrule
0 & all\_spaces\_reto\_grande\_extra\_features & {\cellcolor[HTML]{006837}} \color[HTML]{F1F1F1} 0,95 & {\cellcolor[HTML]{006837}} \color[HTML]{F1F1F1} 0,11 & {\cellcolor[HTML]{006837}} \color[HTML]{F1F1F1} 1,39 \\
1 & reto\_grande\_garage\_extra\_features & {\cellcolor[HTML]{FEEB9D}} \color[HTML]{000000} 5,07 & {\cellcolor[HTML]{D5ED88}} \color[HTML]{000000} 2,05 & {\cellcolor[HTML]{FEE28F}} \color[HTML]{000000} 8,01 \\
2 & reto\_grande\_indoor\_extra\_features & {\cellcolor[HTML]{D22B27}} \color[HTML]{F1F1F1} 7,60 & {\cellcolor[HTML]{FDB163}} \color[HTML]{000000} 3,55 & {\cellcolor[HTML]{EC5C3B}} \color[HTML]{F1F1F1} 10,65 \\
3 & reto\_grande\_outdoor\_extra\_features & {\cellcolor[HTML]{A50026}} \color[HTML]{F1F1F1} 8,27 & {\cellcolor[HTML]{A50026}} \color[HTML]{F1F1F1} 5,06 & {\cellcolor[HTML]{A50026}} \color[H